In [125]:
from preprocess import drop_useless

In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [127]:
data_test=pd.read_csv('../../data/test_airbnb_berlin.xls')
data_train=pd.read_csv('../../data/train_airbnb_berlin.xls')

In [128]:
#Data visualisation

# computing number of rows
rows = len(data_train.axes[0])
print('number row :',rows)
# computing number of columns
cols = len(data_train.axes[1])
print('number columns :',cols)

number row : 15692
number columns : 39


In [129]:
data_train.columns.values

array(['Listing ID', 'Listing Name', 'Host ID', 'Host Name', 'Host Since',
       'Host Response Time', 'Host Response Rate', 'Is Superhost',
       'neighbourhood', 'Neighborhood Group', 'City', 'Postal Code',
       'Country Code', 'Country', 'Latitude', 'Longitude',
       'Is Exact Location', 'Property Type', 'Room Type', 'Accomodates',
       'Bathrooms', 'Bedrooms', 'Beds', 'Square Feet', 'Guests Included',
       'Min Nights', 'Reviews', 'First Review', 'Last Review',
       'Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating',
       'Value Rating', 'Instant Bookable', 'Business Travel Ready',
       'Price'], dtype=object)

In [130]:
data_train.head(1)

,Listing ID,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,19665213.0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0


In [131]:
data_train=drop_useless(data_train)

__Analyse du taux de missing par feature__

In [132]:
#liste des features avec un taux de "missing" moyen (~18%)
listfeatures=[] 

def missing(df):
    total = 0
    for col in df.columns:
        miss = df[col].isnull().sum()
        pct = df[col].isna().mean() * 100
        total += miss
        if miss != 0:
            print('{} => {} [{}%]'.format(col, df[col].isnull().sum(), round(pct, 2)))
            if pct>15 and pct <25 :
                listfeatures.append(col)
        #else:
        #    print('{} => {} '.format(col, 'no missing value'))

    if total == 0:
        print("no missing values")
        
missing(data_train)

Host Response Time => 7075 [45.09%]
Host Response Rate => 7075 [45.09%]
Is Superhost => 23 [0.15%]
Bathrooms => 14 [0.09%]
Bedrooms => 5 [0.03%]
Beds => 8 [0.05%]
Square Feet => 15389 [98.07%]
Overall Rating => 2962 [18.88%]
Accuracy Rating => 2971 [18.93%]
Cleanliness Rating => 2970 [18.93%]
Checkin Rating => 2973 [18.95%]
Communication Rating => 2970 [18.93%]
Location Rating => 2971 [18.93%]
Value Rating => 2972 [18.94%]
Price => 9 [0.06%]


On remarque la présence de quelques astérix dans le dataset: nous choisissons de les remplacer par des NAN.

In [ ]:
from pandas import NA
data_train = data_train.replace("*", NA)

__Gestion des features à taux de "missing" extrême__

Lorsque l'on regarde les features où il manque moins de 30 données, on se rend compte que l'absence de ces données est MCAR. On se permet donc de supprimer les lignes où ces données sont manquantes. 
Il y a une exception sur la feature 'Is Superhost': si pour ces host il y a une note générale au dessus de 80, on les catégorises en Superhost, sinon, non. 

In [133]:
#Suppression des features à taux de missing trop élevé 
data_train=data_train.drop(['Square Feet','Host Response Time','Host Response Rate'], axis=1)

In [134]:
#Listwise deletion pour les features à taux de missing faible (borne max des rows concernés : 60 /15k = 0.4%)
for j in ['Price','Bathrooms','Bedrooms','Beds']:
    for i in data_train[data_train[j].isna()== True].index:
        data_train=data_train.drop(i, axis=0)

In [135]:
# Gestion des 'Is Superhost' : si le host est noté, et ce au dessus de 80, il est supposé superhost, sinon il ne l'est pas
for i in data_train[data_train['Is Superhost'].isna()== True].index:
    if data_train.loc[i,'Overall Rating']>=80:
        data_train.loc[i,'Is Superhost']='t'
    else:
        data_train.loc[i,'Is Superhost']='f'

__Recherche de corrélations entre les features présentant un taux de missing value "NaN" autour de 18%__

On va s'aider de la liste **listfeatures** définie plus tôt


In [136]:
#tableau de correlation de "missing" entre deux features
tablefeatures=np.ones((len(listfeatures),len(listfeatures)))

#calcul des corrélations de missing entre deux features
index1=0 
for q in listfeatures:
    nanrows=len(data_train[data_train[q].isna()== True].axes[0])
    index2=0
    for p in listfeatures :
        if q!=p:
            bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
        #print ("P("+q+" et "+p+" vaut NaN) =" + str(bothmiss/nanrows))
            tablefeatures[index1,index2]=bothmiss/nanrows
        index2+=1
    index1+=1   

#affichage du tableau de corrélations
print("Pourcentage minimal de corrélation entre deux features de la liste : " + str(round(tablefeatures.min()*100,2)) + "% \n")
print(tablefeatures)

<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_tra

Pourcentage minimal de corrélation entre deux features de la liste : 99.63% 

[[1.         1.         1.         1.         1.         1.
  1.        ]
 [0.9969574  1.         0.99966193 1.         0.99966193 0.99966193
  0.99966193]
 [0.99729456 1.         1.         1.         1.         1.
  1.        ]
 [0.99628378 0.99932432 0.99898649 1.         0.99898649 0.99932432
  0.99932432]
 [0.99729456 1.         1.         1.         1.         1.
  1.        ]
 [0.9969574  0.99966193 0.99966193 1.         0.99966193 1.
  1.        ]
 [0.99662048 0.9993241  0.9993241  0.99966205 0.9993241  0.99966205
  1.        ]]


<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_train[p].isna()== True].axes[0])
<ipython-input-136-8558d91beca4>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bothmiss=len(data_train[data_train[q].isna()== True][data_tra

On peut donc raisonnablement affirmer que ces features sont de type MNAR : une listwise deletion n'est donc pas conseillée (et avec 18% d'instances concernées, ce n'est pas idéal)

__Enregistrement__

In [142]:
#fichier csv où l'on sans les ratings 
data_train_clean=data_train.drop(['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating'], axis=1)
data_train_clean.to_csv('../../data/data_train_clean.csv',index=False)
print(missing(data_train_clean))


#fichier csv où l'on garde les ratings

data_train_clean_ratings_included = data_train

for j in ['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating','Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating']:
       for i in data_train_clean_ratings_included[data_train_clean_ratings_included[j].isna()== True].index:
              data_train_clean_ratings_included=data_train_clean_ratings_included.drop(i, axis=0)

data_train_clean_ratings_included.to_csv('../../data/data_train_clean_ratings_included.csv',index=False)
print(missing(data_train_clean_ratings_included))

neighbourhood => 203 [1.3%]
Property Type => 75 [0.48%]
Accomodates => 19 [0.12%]
Bathrooms => 11 [0.07%]
Bedrooms => 11 [0.07%]
Beds => 12 [0.08%]
Guests Included => 10 [0.06%]
Min Nights => 246 [1.57%]
None
neighbourhood => 171 [1.35%]
Property Type => 56 [0.44%]
Accomodates => 17 [0.13%]
Bathrooms => 8 [0.06%]
Bedrooms => 10 [0.08%]
Beds => 10 [0.08%]
Guests Included => 8 [0.06%]
Min Nights => 153 [1.21%]
None
